In [ ]:
!jupyter nbconvert --to notebook --ClearOutputPreprocessor.enabled=True knowledge_graph.ipynb

#Assessment 2: Building a Knowledge Graph with Semantic Search and Query Answering
Instructions:
- Build a basic RDF knowledge graph using the rdflib library.
- Example: "Alice is a person," "Alice knows Bob," "Bob works at Acme Corp."
- Convert the RDF triples into simple text sentences (e.g., "Alice knows Bob").
- Embed these text sentences using any method you prefer (e.g., sentence-transformers, HuggingFace, or other embedding techniques).
- Store the embeddings in FAISS or ChromaDB.
- Accept a simple user query like: "Who does Alice know?" and return the relevant text from the stored embeddings.

In [ ]:
!pip install rdflib sentence-transformers chromadb


In [ ]:
# Build basic RDF
from rdflib import Graph, Namespace, RDF

g = Graph()
ex = Namespace("http://example.org/")

g.add((ex.Alice, RDF.type, ex.Person))
g.add((ex.Alice, ex.knows, ex.Bob))
g.add((ex.Bob, ex.worksAt, ex.AcmeCorp))


In [ ]:
# Convert RDF triples into simple text sentences
sentences = [
    "Alice is a person",
    "Alice knows Bob",
    "Bob works at Acme Corp"
]


In [ ]:
# create Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
sentence_embeddings = model.encode(sentences)


In [ ]:
# Store embeddings in ChromaDB
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="knowledge_graph1")

for i in range(len(sentence_embeddings)):
    collection.add(
        ids=[str(i)],
        embeddings=[sentence_embeddings[i]],
        documents=[sentences[i]]
    )


In [ ]:
# Query
query = "Who does Alice know"
query = query.lower()

query_embedding = model.encode([query])


In [ ]:
# Retrive
result = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

print("Answer:", result["documents"][0][0])
